# 카페 url추출

In [13]:
import pandas as pd
data = pd.read_csv('./성수동 카페.csv')

In [15]:
len(data)

141

## 카페명 확인

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 39 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   상가업소번호     141 non-null    object 
 1   상호명        141 non-null    object 
 2   지점명        2 non-null      object 
 3   상권업종대분류코드  141 non-null    object 
 4   상권업종대분류명   141 non-null    object 
 5   상권업종중분류코드  141 non-null    object 
 6   상권업종중분류명   141 non-null    object 
 7   상권업종소분류코드  141 non-null    object 
 8   상권업종소분류명   141 non-null    object 
 9   표준산업분류코드   141 non-null    object 
 10  표준산업분류명    141 non-null    object 
 11  시도코드       141 non-null    int64  
 12  시도명        141 non-null    object 
 13  시군구코드      141 non-null    int64  
 14  시군구명       141 non-null    object 
 15  행정동코드      141 non-null    int64  
 16  행정동명       141 non-null    object 
 17  법정동코드      141 non-null    int64  
 18  법정동명       141 non-null    object 
 19  지번코드       141 non-null    float64
 20  대지구분코드    

## 상호명 + ' 성수' 추가해서 cafe_names 리스트에 저장
- '성수'를 추가해야 검색이 잘되는 듯

In [43]:
# 상호명 저장
cafe_names = []
for name in data['상호명']:
    name +=' 성수'
    cafe_names.append(name)
print(cafe_names)

['카페메이 성수', '베티버 성수', 'Pauline CoffeeBar 성수', '로우커피스탠드 성수', '훔볼트 성수', '에이투비 Cafe&Bar 성수', '스테치 성수', '309디저트카페 성수', '앤드밀 성수점 성수', 'BNHR 성수', '젤라또투유 성수', '마를리 성수', '커피코코 성수', '치카치카 성수', '엣모스피어 성수', '어니언 성수 성수', '레이더 성수 성수', '쎈느Scene 성수', '카페크라비 성수', '아이오쓰리 에이스하이엔드점 성수', '라니커피 성수', '카멜커피 성수점 성수', '스프레이하우스 성수', '단과자 성수', '럭럭카페 성수', '야페 성수', '프레이커피바 성수', '더하나누 성수', '아크밸리서비스 성수', '러프러프 성수', '얌커피 성수', '스펑크커피 성수', '스트랏 성수', '오프커피 성수', '카페엘든 성수', '크레송 성수', '업.사이드 성수', '커피나인컴퍼니 성수', '카페,바로 성수', '성수도시개발 성수', '어퍼룸 성수', '그레이박스카페 성수', '성수동대림창고갤러리 성수', '유스트레스 성수', '리브릭 성수', '트리운드 성수', '커피베이 성수', '턴온 성수', '굴림 성수', '하루앤원데이 성수', '플록스phlox 성수', '반디자인 성수', '뿐또블루 성수', '아쿠아산타성수카페 성수', '어페어커피 성수', '진혁 성수', '뚝도청춘상인회협동조합 성수', '앙골로에스프레소 성수', '커피식구성수센터 성수', '카페페를 성수', '창창커피 성수', '카페포제 성수', '어커 성수', '온유 성수', '차일디쉬 성수', '오푸 성수', '모센즈스위트 성수', '어피스앤드피스 성수', '에이치비씨 성수', '선샤인비치 성수', '베러초이스 성수', '에젤커피 성수', '라에비뉴 성수', '나드에프앤비 성수', '에이테르 성수', '카페시벳 성수', '리얼 성수', '로우키 성수', '피카앤코피카성 성수', '티앤에이 성수', '니드 성수', '힘이나커피 성수', '높은

## 상호명을 네이버에 검색해가면서 네이버플레이스 url추출
- 첫 코드는 BeautifulSoup사용 -> HTTP에러 남 (이게 더 빠르긴 해서 나중에 쓸 수 있다면 쓰면 좋을 듯)
- 두번째 코드는 webdriver사용 (느린 대신 에러는 안남) 

In [41]:
import requests
from bs4 import BeautifulSoup
import re
#place-app-root
# 결과를 저장할 리스트
url_list = []
id_list = []
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'
}


for cafe_name in cafe_names:
    # 검색 URL 생성
    search_url = f"https://m.search.naver.com/search.naver?sm=mtp_hty.top&where=m&query={cafe_name}"
    
    # 페이지 가져오기
    response = requests.get(search_url, headers=headers)
    response.raise_for_status()  # 요청이 실패하면 예외 발생
    
    # HTML 파싱
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 루트 셀렉터 내의 모든 a 태그 가져오기
    root_element = soup.select_one('#place-app-root')
    found_url = None

    if root_element:
        links = root_element.find_all('a', href=True)
        for link in links:
            href = link['href']
            if href.startswith('https://m.place.naver.com/restaurant/'):
                found_url = href
                break  # 첫 번째로 찾은 URL만 사용
    
    # 첫 번째 셀렉터에서 찾지 못한 경우, 두 번째 셀렉터 시도
    if not found_url:
        secondary_root_element = soup.select_one('#loc-main-section-root > div')
        if secondary_root_element:
            links = secondary_root_element.find_all('a', href=True)
            for link in links:
                href = link['href']
                if href.startswith('https://m.place.naver.com/restaurant/'):
                    found_url = href
                    break  # 첫 번째로 찾은 URL만 사용
    
    # URL에서 {} 부분 추출
    if found_url:
        match = re.search(r'https://m\.place\.naver\.com/restaurant/([^/]+)/(/.*)?', found_url)
        if match:
            restaurant_id = match.group(1)
            id_list.append(restaurant_id)
        else:
            restaurant_id = None
            id_list.append(None)
    else:
        id_list.append(None)

    # 결과 출력 (매번 하나씩 확인 가능)
    print(f"카페명: {cafe_name}, Restaurant ID: {restaurant_id if found_url else 'None'}, url: {found_url}")

    # 결과 리스트에 추가
    url_list.append(found_url)

# 결과 확인
# for cafe, url in zip(cafe_names, url_list):
#     print(f"카페명: {cafe}, URL: {url}")

    #loc-main-section-root > div

카페명: 카페메이 성수, Restaurant ID: None, url: None
카페명: 베티버 성수, Restaurant ID: 1383256753, url: https://m.place.naver.com/restaurant/1383256753/photo
카페명: Pauline CoffeeBar 성수, Restaurant ID: 1215237779, url: https://m.place.naver.com/restaurant/1215237779/photo?entry=plt
카페명: 로우커피스탠드 성수, Restaurant ID: 1168698330, url: https://m.place.naver.com/restaurant/1168698330/photo?entry=plt
카페명: 훔볼트 성수, Restaurant ID: 1504347953, url: https://m.place.naver.com/restaurant/1504347953/photo?entry=plt
카페명: 에이투비 Cafe&Bar 성수, Restaurant ID: 1537971823, url: https://m.place.naver.com/restaurant/1537971823/booking?theme=place&entry=pll
카페명: 스테치 성수, Restaurant ID: 1180418614, url: https://m.place.naver.com/restaurant/1180418614/photo?entry=plt
카페명: 309디저트카페 성수, Restaurant ID: 1691420621, url: https://m.place.naver.com/restaurant/1691420621/booking?theme=place&entry=pll
카페명: 앤드밀 성수점 성수, Restaurant ID: 1943210266, url: https://m.place.naver.com/restaurant/1943210266/location?entry=plt
카페명: BNHR 성수, Restaurant 

HTTPError: 403 Client Error: Forbidden for url: https://m.search.naver.com/search.naver?sm=mtp_hty.top&where=m&query=%EC%84%B1%EC%88%98%EB%8F%84%EC%8B%9C%EA%B0%9C%EB%B0%9C%20%EC%84%B1%EC%88%98

In [45]:
# webdriver사용


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import re
import time


# 결과를 저장할 리스트
url_list = []
id_list = []

# Chrome WebDriver 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

for cafe_name in cafe_names:
    # 검색 URL 생성
    search_url = f"https://m.search.naver.com/search.naver?sm=mtp_hty.top&where=m&query={cafe_name}"
    
    # 페이지 로드
    driver.get(search_url)
    time.sleep(0.5)  # 페이지 로드 시간을 고려하여 잠시 대기
    
    # 루트 셀렉터 내의 모든 a 태그 가져오기
    found_url = None
    try:
        root_element = driver.find_element(By.CSS_SELECTOR, '#place-app-root')
        links = root_element.find_elements(By.TAG_NAME, 'a')
        for link in links:
            href = link.get_attribute('href')
            if href.startswith('https://m.place.naver.com/restaurant/'):
                found_url = href
                break  # 첫 번째로 찾은 URL만 사용
    
        # 첫 번째 셀렉터에서 찾지 못한 경우, 두 번째 셀렉터 시도
        if not found_url:
            secondary_root_element = driver.find_element(By.CSS_SELECTOR, '#loc-main-section-root > div')
            links = secondary_root_element.find_elements(By.TAG_NAME, 'a')
            for link in links:
                href = link.get_attribute('href')
                if href.startswith('https://m.place.naver.com/restaurant/'):
                    found_url = href
                    break  # 첫 번째로 찾은 URL만 사용
    
        # URL에서 {} 부분 추출
        if found_url:
            match = re.search(r'https://m\.place\.naver\.com/restaurant/([^/]+)/(/.*)?', found_url)
            if match:
                restaurant_id = match.group(1)
                id_list.append(restaurant_id)
            else:
                restaurant_id = None
                id_list.append(None)
        else:
            id_list.append(None)

        # 결과 출력 (매번 하나씩 확인 가능)
        print(f"카페명: {cafe_name}, Restaurant ID: {restaurant_id if found_url else 'None'}, URL: {found_url}")

        # 결과 리스트에 추가
        url_list.append(found_url)
    
    except Exception as e:
        print(f"카페명: {cafe_name} 오류 발생")
        id_list.append(None)
        url_list.append(None)

# 모든 작업이 끝난 후 브라우저 닫기
driver.quit()

# 최종 결과 확인
for cafe, rest_id in zip(cafe_names, id_list):
    print(f"카페명: {cafe}, Restaurant ID: {rest_id}, URL: {url_list[cafe_names.index(cafe)]}")


카페명: 카페메이 성수, Restaurant ID: None, URL: None
카페명: 베티버 성수, Restaurant ID: 1383256753, URL: https://m.place.naver.com/restaurant/1383256753/photo
카페명: Pauline CoffeeBar 성수, Restaurant ID: 1215237779, URL: https://m.place.naver.com/restaurant/1215237779/photo?entry=plt
카페명: 로우커피스탠드 성수, Restaurant ID: 1168698330, URL: https://m.place.naver.com/restaurant/1168698330/photo?entry=plt
카페명: 훔볼트 성수, Restaurant ID: 1504347953, URL: https://m.place.naver.com/restaurant/1504347953/photo?entry=plt
카페명: 에이투비 Cafe&Bar 성수, Restaurant ID: 1537971823, URL: https://m.place.naver.com/restaurant/1537971823/booking?theme=place&entry=pll
카페명: 스테치 성수, Restaurant ID: 1180418614, URL: https://m.place.naver.com/restaurant/1180418614/photo?entry=plt
카페명: 309디저트카페 성수, Restaurant ID: 1691420621, URL: https://m.place.naver.com/restaurant/1691420621/booking?theme=place&entry=pll
카페명: 앤드밀 성수점 성수, Restaurant ID: 1943210266, URL: https://m.place.naver.com/restaurant/1943210266/location?entry=plt
카페명: BNHR 성수, Restaurant 

카페명: 컴퍼니맨 성수 오류 발생
카페명: 투달러스커피 성수, Restaurant ID: 75758743, URL: https://m.place.naver.com/restaurant/75758743/photo?entry=plt
카페명: Cafelashower 성수, Restaurant ID: None, URL: None
카페명: 카페아모렘 성수 오류 발생
카페명: 예빈당 성수, Restaurant ID: 11861121, URL: https://m.place.naver.com/restaurant/11861121/photo?entry=plt
카페명: 피어커피 성수, Restaurant ID: 33704135, URL: https://m.place.naver.com/restaurant/33704135/photo?entry=plt
카페명: 제이큐리아서티 성수 오류 발생
카페명: 샵힘이나는커피생활 성수 오류 발생
카페명: 왕다방 성수 오류 발생
카페명: 브루잉세레모니 성수, Restaurant ID: 1613470297, URL: https://m.place.naver.com/restaurant/1613470297/photo?entry=plt
카페명: 나루데이타 성수, Restaurant ID: None, URL: None
카페명: 이트앤드링크 성수, Restaurant ID: 1483166635, URL: https://m.place.naver.com/restaurant/1483166635/photo
카페명: 아이오쓰리에이스하이앤드점 성수 오류 발생
카페명: 성수정원 성수, Restaurant ID: 1502355065, URL: https://m.place.naver.com/restaurant/1502355065/booking?theme=place&entry=pll
카페명: 마이쥬스 성수, Restaurant ID: 1626027648, URL: https://m.place.naver.com/restaurant/1626027648/photo?entry=plt
카페

## 검색이 안되는 카페 추출 후 노가다로 넣어주기
- missing_idx에 검색 안되는 카페의 인덱스 저장해줌 

In [73]:
# 카페 이름이 담겨있는 리스트
cafe_names

['카페메이 성수',
 '베티버 성수',
 'Pauline CoffeeBar 성수',
 '로우커피스탠드 성수',
 '훔볼트 성수',
 '에이투비 Cafe&Bar 성수',
 '스테치 성수',
 '309디저트카페 성수',
 '앤드밀 성수점 성수',
 'BNHR 성수',
 '젤라또투유 성수',
 '마를리 성수',
 '커피코코 성수',
 '치카치카 성수',
 '엣모스피어 성수',
 '어니언 성수 성수',
 '레이더 성수 성수',
 '쎈느Scene 성수',
 '카페크라비 성수',
 '아이오쓰리 에이스하이엔드점 성수',
 '라니커피 성수',
 '카멜커피 성수점 성수',
 '스프레이하우스 성수',
 '단과자 성수',
 '럭럭카페 성수',
 '야페 성수',
 '프레이커피바 성수',
 '더하나누 성수',
 '아크밸리서비스 성수',
 '러프러프 성수',
 '얌커피 성수',
 '스펑크커피 성수',
 '스트랏 성수',
 '오프커피 성수',
 '카페엘든 성수',
 '크레송 성수',
 '업.사이드 성수',
 '커피나인컴퍼니 성수',
 '카페,바로 성수',
 '성수도시개발 성수',
 '어퍼룸 성수',
 '그레이박스카페 성수',
 '성수동대림창고갤러리 성수',
 '유스트레스 성수',
 '리브릭 성수',
 '트리운드 성수',
 '커피베이 성수',
 '턴온 성수',
 '굴림 성수',
 '하루앤원데이 성수',
 '플록스phlox 성수',
 '반디자인 성수',
 '뿐또블루 성수',
 '아쿠아산타성수카페 성수',
 '어페어커피 성수',
 '진혁 성수',
 '뚝도청춘상인회협동조합 성수',
 '앙골로에스프레소 성수',
 '커피식구성수센터 성수',
 '카페페를 성수',
 '창창커피 성수',
 '카페포제 성수',
 '어커 성수',
 '온유 성수',
 '차일디쉬 성수',
 '오푸 성수',
 '모센즈스위트 성수',
 '어피스앤드피스 성수',
 '에이치비씨 성수',
 '선샤인비치 성수',
 '베러초이스 성수',
 '에젤커피 성수',
 '라에비뉴 성수',
 '나드에프앤비 성수',
 '에이테르 성수',
 '카페시

In [46]:
# id가 None인 카페에 대해 cafe_names리스트에서의 인덱스와 카페이름 반환

missing_idx = []
for idx, cafe_id in enumerate(id_list):
    if cafe_id == None:
        missing_idx.append(idx)
        print(f'idx: {idx} , cafe : {cafe_names[idx]}')

print(id_list)


idx: 0 , cafe : 카페메이 성수
idx: 14 , cafe : 엣모스피어 성수
idx: 21 , cafe : 카멜커피 성수점 성수
idx: 27 , cafe : 더하나누 성수
idx: 28 , cafe : 아크밸리서비스 성수
idx: 30 , cafe : 얌커피 성수
idx: 31 , cafe : 스펑크커피 성수
idx: 32 , cafe : 스트랏 성수
idx: 33 , cafe : 오프커피 성수
idx: 34 , cafe : 카페엘든 성수
idx: 36 , cafe : 업.사이드 성수
idx: 37 , cafe : 커피나인컴퍼니 성수
idx: 38 , cafe : 카페,바로 성수
idx: 39 , cafe : 성수도시개발 성수
idx: 45 , cafe : 트리운드 성수
idx: 46 , cafe : 커피베이 성수
idx: 47 , cafe : 턴온 성수
idx: 50 , cafe : 플록스phlox 성수
idx: 51 , cafe : 반디자인 성수
idx: 52 , cafe : 뿐또블루 성수
idx: 55 , cafe : 진혁 성수
idx: 56 , cafe : 뚝도청춘상인회협동조합 성수
idx: 61 , cafe : 카페포제 성수
idx: 63 , cafe : 온유 성수
idx: 68 , cafe : 에이치비씨 성수
idx: 69 , cafe : 선샤인비치 성수
idx: 74 , cafe : 에이테르 성수
idx: 76 , cafe : 리얼 성수
idx: 77 , cafe : 로우키 성수
idx: 78 , cafe : 피카앤코피카성 성수
idx: 79 , cafe : 티앤에이 성수
idx: 80 , cafe : 니드 성수
idx: 81 , cafe : 힘이나커피 성수
idx: 83 , cafe : 라스베이글 성수
idx: 84 , cafe : 마르코스 성수
idx: 85 , cafe : 한강커피숍 성수
idx: 88 , cafe : 한화비앤비 성수
idx: 93 , cafe : Webconey 성수
idx: 94 , cafe : 카페수다 성수

In [81]:
# 혹시 모르니 카피해두기
id_list_copy = id_list.copy()

In [85]:
# 카페id가 없는 카페들에 대해 직접 id 찾아옴

missing_idx_update=['1210627765','1267437086','55585656', None, '1261617871', '1067014011', None, '1410613160', '1921874344',None, 
                   '1259420360', None, '1916513487',  None, None, '1573389383','1348444139', '1515867867', None, None , None, None, None, 
                   '1169957780', None, None, None, '1190505473','1393890643',None, None, '1935137830', None, '1173495456', None, None,
                   None, None, '34351850', '1846703943',None, None, '1590690124', '1403886445', None, None, None, None, '1623328599',
                   None, None, None, None, '1273097610','1824108106', None, None, None, None, None ]

In [86]:
# missing개수랑 맞는지 확인
len(missing_idx_update)

60

In [93]:
# id_list 업데이트 해주기 

for idx, missingidx in enumerate(missing_idx):
    id_list[missingidx] = missing_idx_update[idx]



print(id_list)

['1210627765', '1383256753', '1215237779', '1168698330', '1504347953', '1537971823', '1180418614', '1691420621', '1943210266', '1223354536', '1247801780', '742676592', '1853990672', '37191703', '1267437086', '38561949', '1525007973', '1030871168', '1872719083', '1623328599', '1227111236', '55585656', '1300253897', '1177180342', '1168976634', '1701590225', '1852410455', None, '1261617871', '1829908685', '1067014011', None, '1410613160', '1921874344', None, '37392409', '1259420360', None, '1916513487', None, '1964635228', '1949527561', '37910590', '1999678821', '1424869222', None, '1573389383', '1348444139', '1397836032', '38692829', '1515867867', None, None, '1517487358', '1863706992', None, None, '1066610472', '1447656721', '1362812488', '1005128677', None, '1560084717', '1169957780', '1217287130', '1127892000', '1709396208', '1447612432', None, None, '1096462943', '37683876', '1199055314', '30999464', None, '20604985', '1190505473', '1393890643', None, None, '1935137830', None, '10810

In [97]:
# id가 있는 카페에 대해 딕셔너리 만들기

cafe_dict = {}
for idx, cafe_id in enumerate(id_list):
    if cafe_id:
        cafe_dict[f'{cafe_names[idx]}'] = cafe_id
print(cafe_dict)

{'카페메이 성수': '1210627765', '베티버 성수': '1383256753', 'Pauline CoffeeBar 성수': '1215237779', '로우커피스탠드 성수': '1168698330', '훔볼트 성수': '1504347953', '에이투비 Cafe&Bar 성수': '1537971823', '스테치 성수': '1180418614', '309디저트카페 성수': '1691420621', '앤드밀 성수점 성수': '1943210266', 'BNHR 성수': '1223354536', '젤라또투유 성수': '1247801780', '마를리 성수': '742676592', '커피코코 성수': '1853990672', '치카치카 성수': '37191703', '엣모스피어 성수': '1267437086', '어니언 성수 성수': '38561949', '레이더 성수 성수': '1525007973', '쎈느Scene 성수': '1030871168', '카페크라비 성수': '1872719083', '아이오쓰리 에이스하이엔드점 성수': '1623328599', '라니커피 성수': '1227111236', '카멜커피 성수점 성수': '55585656', '스프레이하우스 성수': '1300253897', '단과자 성수': '1177180342', '럭럭카페 성수': '1168976634', '야페 성수': '1701590225', '프레이커피바 성수': '1852410455', '아크밸리서비스 성수': '1261617871', '러프러프 성수': '1829908685', '얌커피 성수': '1067014011', '스트랏 성수': '1410613160', '오프커피 성수': '1921874344', '크레송 성수': '37392409', '업.사이드 성수': '1259420360', '카페,바로 성수': '1916513487', '어퍼룸 성수': '1964635228', '그레이박스카페 성수': '1949527561', '성수동대림창고갤러리 성수': '3791059

In [104]:
확인
len(cafe_dict)

105

In [105]:
cafe_dict

{'카페메이 성수': '1210627765',
 '베티버 성수': '1383256753',
 'Pauline CoffeeBar 성수': '1215237779',
 '로우커피스탠드 성수': '1168698330',
 '훔볼트 성수': '1504347953',
 '에이투비 Cafe&Bar 성수': '1537971823',
 '스테치 성수': '1180418614',
 '309디저트카페 성수': '1691420621',
 '앤드밀 성수점 성수': '1943210266',
 'BNHR 성수': '1223354536',
 '젤라또투유 성수': '1247801780',
 '마를리 성수': '742676592',
 '커피코코 성수': '1853990672',
 '치카치카 성수': '37191703',
 '엣모스피어 성수': '1267437086',
 '어니언 성수 성수': '38561949',
 '레이더 성수 성수': '1525007973',
 '쎈느Scene 성수': '1030871168',
 '카페크라비 성수': '1872719083',
 '아이오쓰리 에이스하이엔드점 성수': '1623328599',
 '라니커피 성수': '1227111236',
 '카멜커피 성수점 성수': '55585656',
 '스프레이하우스 성수': '1300253897',
 '단과자 성수': '1177180342',
 '럭럭카페 성수': '1168976634',
 '야페 성수': '1701590225',
 '프레이커피바 성수': '1852410455',
 '아크밸리서비스 성수': '1261617871',
 '러프러프 성수': '1829908685',
 '얌커피 성수': '1067014011',
 '스트랏 성수': '1410613160',
 '오프커피 성수': '1921874344',
 '크레송 성수': '37392409',
 '업.사이드 성수': '1259420360',
 '카페,바로 성수': '1916513487',
 '어퍼룸 성수': '1964635228',
 '그레이박스카페 성수': '19

In [112]:
# 이름 바꿔줘야하는게 있어서 바꿔줌
cafe_dict["기와르"] = cafe_dict.pop("성수한옥 성수")

In [116]:
cafe_df = pd.DataFrame(list(cafe_dict.items()), columns=['카페명', 'id값'])
cafe_df

,카페명,id값
0,카페메이 성수,1210627765
1,베티버 성수,1383256753
2,Pauline CoffeeBar 성수,1215237779
3,로우커피스탠드 성수,1168698330
4,훔볼트 성수,1504347953
...,...,...
100,카페소림 성수,1194892405
101,커피냅로스터스성수 성수,1355804064
102,도렐 성수점 성수,1432336976
103,오우드 성수 성수,1216172292


In [124]:
cafe_df.to_json('카페_최종.json',force_ascii=False)

# 네이버 크롤링 : 카페 리뷰 추출

In [64]:
from selenium.webdriver.common.by import By
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import datetime
import requests
import json

# 모든 카페 리뷰를 담을 리스트 
all_reviews_data = []

# 카페 아이디를 url에 추가하면서 돌기 (추천순 리뷰로!!)
for idx, cafe_id in enumerate(id_list[1:6]):
    # 카페명
    cafe_name = cafe_names[idx]
    print(f'{idx} : {cafe_name}')

    # url
    url = f'https://m.place.naver.com/restaurant/{cafe_id}/review/visitor?entry=ple&review'

    # Webdriver headless mode setting
    options = webdriver.ChromeOptions()
    # options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    # options.add_argument("disable-gpu")

    # BS4 setting for secondary access
    session = requests.Session()
    headers = {
        "User-Agent": "user value"}

    retries = Retry(total=5,
                    backoff_factor=0.1,
                    status_forcelist=[500, 502, 503, 504])

    session.mount('http://', HTTPAdapter(max_retries=retries))

    # New dictionary to store reviews
    reviews_data = []

    # Start crawling/scraping!
    try:
        # driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options)
        driver = webdriver.Chrome()
        res = driver.get(url)
        driver.implicitly_wait(5)

        # Pagedown
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)

        try:
            for i in range(30):  # 최대 30번 반복
                driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').click()
                time.sleep(5)
        except Exception as e:
            print(f'finish {e}')
        else:
            print('30times_finish')
        try:
            # 줄인 대기 시간, 명시적 대기로 대체 
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'li.pui__X35jYm.EjjAW'))
            )
        except Exception as e:
            print(f"Element not found or timeout for cafe_name: {cafe_name}")
            continue
            
        print('진행중..')
        
        html = driver.page_source
        bs = BeautifulSoup(html, 'html.parser')
        reviews = bs.select('li.pui__X35jYm.EjjAW')

        for r in reviews:


            # nickname
            nickname = r.select_one('div.pui__JiVbY3 > span.pui__uslU0d')

            # content
            content = r.select_one('div.pui__vn15t2 > a.pui__xtsQN-')

            # date
            date_elements = r.select('div.pui__QKE5Pr > span.pui__gfuUIT > time')
            date = date_elements[0].text if date_elements else 'N/A'

            # revisit
            revisit_span = r.select('div.pui__QKE5Pr > span.pui__gfuUIT')
            revisit = revisit_span[1].text if len(revisit_span) > 1 else 'N/A'

            # exception handling
            nickname = nickname.text if nickname else ''
            content = content.text if content else None
            date = date if date else ''
            revisit = revisit if revisit else ''
            time.sleep(0.06)

            # Append the review to the list
            if content:
                reviews_data.append({
                    'cafe name': cafe_name,
                    'url' : url,
                    'nickname': nickname,
                    'content': content,
                    'date': date,
                    'revisit': revisit
                })

            time.sleep(0.06)

        # Append the current cafe's reviews to the main list
        all_reviews_data.append({
            'cafe name': cafe_name,
            'reviews': reviews_data
        })

        # Save the file
#         now = datetime.datetime.now()
#         file_name = 'naver_review_' + '.json'
#         with open(file_name, 'w', encoding='utf-8') as f:
#             json.dump(reviews_data, f, ensure_ascii=False, indent=4)

    except Exception as e:
        print(f"An error occurred for cafe_name: {cafe_name}, error:{e}")
        
    finally:
        driver.quit()

# Save the file 
file_name = 'naver_review_5개' + '.json'
with open(file_name, 'w', encoding='utf-8') as f:
    json.dump(all_reviews_data, f, ensure_ascii=False, indent=4)


0 : 카페메이 성수
30times_finish
Element not found or timeout for cafe_name: 카페메이 성수
1 : 베티버 성수
finish Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a"}
  (Session info: chrome=127.0.6533.100); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000102dc10b8 cxxbridge1$str$ptr + 1887276
1   chromedriver                        0x0000000102db9794 cxxbridge1$str$ptr + 1856264
2   chromedriver                        0x00000001029c882c cxxbridge1$string$len + 88524
3   chromedriver                        0x0000000102a0c834 cxxbridge1$string$len + 367060
4   chromedriver                        0x0000000102a4448c cxxbridge1$string$len + 595500
5   chromedriver                        0x0000000102a01474 cxxbridge1$string$len + 321044
6   chromedriver    

KeyboardInterrupt: 

In [59]:
all_reviews_data

[]

In [54]:
naverdata = pd.read_json('./naver_review_10개.json')

In [55]:
naverdata

""


In [66]:
id_list.to_json('추출된카페id.json', orient='records', lines=True)

AttributeError: 'list' object has no attribute 'to_json'

In [67]:
with open('카페id추출.json', 'w') as json_file:
    json.dump(id_list, json_file, indent=4)  # indent=4는 파일을 읽기 쉽게 들여쓰기 합니다.

In [69]:

pd.set_option('display.float_format', '{:.0f}'.format)
print(pd.read_json('./카페id추출.json'))

             0
0          NaN
1   1383256753
2   1215237779
3   1168698330
4   1504347953
..         ...
136 1612076914
137 1194892405
138 1355804064
139 1432336976
140 1216172292

[141 rows x 1 columns]
